In [31]:
import tushare as ts
import pandas as pd
from datetime import datetime
import os.path
import time
import numpy as np

In [32]:
def update_data_or_not(fileName, year):
    import os
    yearEnd = datetime.now().year-1  #去年
    updateTime = 0
    if ((year==yearEnd and datetime.now().month >= 5) or year < yearEnd):
        updateTime = 30
    else:
        updateTime = 1
    updateTime = updateTime*24*60*60
    finfo = os.stat(fileName)
    return(( time.time() - finfo.st_mtime) > updateTime)
    

In [33]:
def get_data(dataType, year, q=4): #年报
    typeAll = {'profit', 'report', 'growth'}
    if (dataType not in typeAll):
        print('%s not in %s' %(type, typeAll,))
        return
    saveFileName = './data/%s%02d%s.csv' % (year,q, dataType)
    if os.path.isfile(saveFileName):
        df = pd.read_csv(saveFileName)#, encoding='gb18030')
        # return df
    if (update_data_or_not(saveFileName, year)):  # 如果需要更新
        fun = eval('ts.get_%s_data' % dataType)
        dfnew = fun(year, q)
        df = pd.concat([df[~df['code'].isin(dfnew['code'].values)],dfnew])
        df.to_csv(saveFileName, index=False, encoding='utf-8')
    return df


In [34]:
dfs = {'profit':{}, 'report':{}, 'growth':{}}
types = {'profit', 'report', 'growth'}
yearEnd = datetime.now().year  #今年
for t in types:
    for y in range(yearEnd-5, yearEnd):
        dfs[t][y] = get_data(t, y).rename(columns={i: i+str(y-yearEnd+5) for i in get_data(t, y).columns if i not in {'code'}})
        #dfs[t][y]['code'] = dfs[t][y]['code'].astype(int)
        dfs[t][y]['code'] = dfs[t][y]['code'].apply(lambda x: '%06d' % int(x))
#         
#             

In [35]:
dfs_growth = pd.DataFrame(columns=['code'])
for i in range(yearEnd-5, yearEnd):
    dfs_growth = dfs_growth.merge(dfs['growth'][i], on=['code'], how='outer')

# dfs['profit'][2013].merge(dfs['profit'][2014], on=['code', 'name'], how='outer')

In [36]:
stockGDF = dfs_growth[(dfs_growth.nprg0>25)&(dfs_growth.nprg1>25)\
           &(dfs_growth.nprg2>25)&(dfs_growth.nprg3>25)]


In [37]:
price = ts.get_realtime_quotes(stockGDF['code'])
def real_price(x, y):
    return x if x > 0 else y

price['price'] = price['price'].astype(float)
price['pre_close'] = price['pre_close'].astype(float)
price['price_real'] = price.apply(lambda x: real_price(x['price'], x['pre_close']), axis=1)
#price['price_true'] = map(lambda x, y : , price['price'].astype(float), price['pre_close'].astype(float))
#price = price.drop(['price_true'], axis=1)

In [38]:
stockGDF = stockGDF.merge(price, on='code')

In [39]:
dfs_report = pd.DataFrame(columns=['code'])
for i in range(yearEnd-2, yearEnd):
    dfs_report = dfs_report.merge(dfs['report'][i], on=['code'], how='outer')
# dfs['profit'][2013].merge(dfs['profit'][2014], on=['code', 'name'], how='outer')
dfs_report = dfs_report.drop_duplicates()
dfs_report['eps'] = dfs_report.apply(lambda x: x['eps3'] if pd.isnull(x['eps4']) else x['eps4'], axis=1)
dfs_report['rec_report_date'] = dfs_report.apply(lambda x: '%d-%s' % (yearEnd-1,x['report_date3'])\
                                                 if pd.isnull(x['report_date4']) else '%d-%s' % (yearEnd,x['report_date4']), axis=1)

In [40]:

stockGDF = stockGDF.merge(dfs_report, on='code', how='left')

In [41]:
#stockGDF[stockGDF.code == '002078']

In [42]:
stockGDF['nprg'] = stockGDF.apply(lambda x: x['nprg3'] if pd.isnull(x['nprg4']) else x['nprg4'], axis=1)

In [43]:
df_basics = ts.get_stock_basics().drop(['name'], axis=1)
stockGDF = stockGDF.merge(df_basics, left_on='code', right_index=True)

In [44]:
stockGDF['peg'] = round(stockGDF['pe']/stockGDF['nprg'],2)

In [45]:
stockGDF[['code', 'name','peg', 'pe', 'eps', 'nprg', 'price_real', 'rec_report_date']\
         + ['nprg'+str(i) for i in range(5)]][stockGDF.peg < 0.4]

,code,name,peg,pe,eps,nprg,price_real,rec_report_date,nprg0,nprg1,nprg2,nprg3,nprg4
6,600816,安信信托,0.21,15.80,1.71,76.1722,12.90,2017-01-25,172.7590,266.0663,68.2561,76.1722,NaN
10,002019,亿帆医药,0.24,21.07,0.64,88.8034,20.70,2017-03-31,126.2480,829.1151,52.1242,88.8034,NaN
23,300237,美晨生态,0.20,23.41,0.55,115.0827,13.19,2017-04-25,61.6673,237.5191,106.3751,115.0827,NaN
25,002841,视源股份,0.29,31.68,2.04,109.2521,63.59,2017-04-01,57.5867,37.6380,56.9934,109.2521,NaN
28,300244,迪安诊断,0.39,35.03,0.48,89.0256,21.00,2017-03-28,51.5895,42.0045,39.8372,89.0256,NaN
31,300253,卫宁健康,0.32,72.64,0.64,225.3914,8.39,2017-04-25,49.4068,56.4341,31.6772,225.3914,NaN
32,300253,卫宁健康,0.32,72.64,0.64,225.3914,8.39,2017-04-25,49.4068,56.4341,31.6772,225.3914,NaN
34,002078,太阳纸业,0.30,16.21,0.42,53.3837,11.34,2017-04-25,44.9830,72.0237,38.0570,53.3837,NaN
42,300296,利亚德,0.37,37.31,0.87,100.6575,19.67,2017-03-28,37.8114,104.1679,102.7194,100.6575,NaN
45,300121,阳谷华泰,0.08,22.87,0.56,298.7958,13.49,2017-02-24,34.3584,71.0576,45.7954,298.7958,NaN


In [48]:
def price_after_report(date, code):
    return ts.get_k_data(code, ktype='D', start=date).iloc[0]['close']
stockGDF['price_after_report'] = stockGDF.apply(lambda x: \
                                                price_after_report(x['rec_report_date'], x['code'])\
                                               , axis=1)

In [57]:
stockGDF[['code', 'name','peg', 'pe', 'eps', 'nprg', 'price_real', 'rec_report_date', 'price_after_report', '涨幅']\
         + ['nprg'+str(i) for i in range(5)]][stockGDF.peg < 0.4]

,code,name,peg,pe,eps,nprg,price_real,rec_report_date,price_after_report,涨幅,nprg0,nprg1,nprg2,nprg3,nprg4
6,600816,安信信托,0.21,15.80,1.71,76.1722,12.90,2017-01-25,11.080,0.164260,172.7590,266.0663,68.2561,76.1722,NaN
10,002019,亿帆医药,0.24,21.07,0.64,88.8034,20.70,2017-03-31,16.747,0.236042,126.2480,829.1151,52.1242,88.8034,NaN
23,300237,美晨生态,0.20,23.41,0.55,115.0827,13.19,2017-04-25,16.951,-0.221875,61.6673,237.5191,106.3751,115.0827,NaN
25,002841,视源股份,0.29,31.68,2.04,109.2521,63.59,2017-04-01,94.570,-0.327588,57.5867,37.6380,56.9934,109.2521,NaN
28,300244,迪安诊断,0.39,35.03,0.48,89.0256,21.00,2017-03-28,31.391,-0.331018,51.5895,42.0045,39.8372,89.0256,NaN
31,300253,卫宁健康,0.32,72.64,0.64,225.3914,8.39,2017-04-25,8.842,-0.051120,49.4068,56.4341,31.6772,225.3914,NaN
32,300253,卫宁健康,0.32,72.64,0.64,225.3914,8.39,2017-04-25,8.842,-0.051120,49.4068,56.4341,31.6772,225.3914,NaN
34,002078,太阳纸业,0.30,16.21,0.42,53.3837,11.34,2017-04-25,6.962,0.628842,44.9830,72.0237,38.0570,53.3837,NaN
42,300296,利亚德,0.37,37.31,0.87,100.6575,19.67,2017-03-28,19.925,-0.012798,37.8114,104.1679,102.7194,100.6575,NaN
45,300121,阳谷华泰,0.08,22.87,0.56,298.7958,13.49,2017-02-24,18.043,-0.252342,34.3584,71.0576,45.7954,298.7958,NaN


In [51]:
stockGDF['涨幅'] = (stockGDF['price_real'] - stockGDF['price_after_report'])/stockGDF['price_after_report']

In [54]:
stockGDF[['code', 'name','涨幅']\
         ][stockGDF.peg < 0.4]

,code,name,涨幅
6,600816,安信信托,0.164260
10,002019,亿帆医药,0.236042
23,300237,美晨生态,-0.221875
25,002841,视源股份,-0.327588
28,300244,迪安诊断,-0.331018
31,300253,卫宁健康,-0.051120
32,300253,卫宁健康,-0.051120
34,002078,太阳纸业,0.628842
42,300296,利亚德,-0.012798
45,300121,阳谷华泰,-0.252342


In [61]:
dfs_growth[['nprg'+str(i) for i in range(5)]][dfs_growth.code == '600522']

,nprg0,nprg1,nprg2,nprg3,nprg4
1181,23.8568,6.8143,70.9107,57.8517,NaN
